In [ ]:
import pandas as pd
import requests

In [ ]:
#---------------------------------------------------
#
# Config
#
#---------------------------------------------------

# This file should be automatically generated from "1. Gaze Projections and AOIs Mapping"
GAZE_DATA_WITH_AOIS_FILE = "data/gazeDataWithAOIs.csv"

In [ ]:
#----------------------------------------------------------------------------------------
#
#
# 1. Oculomotor event detection (i.e., Fixations and saccades)
# 
#
#----------------------------------------------------------------------------------------

In [ ]:
# Read gaze data with AOIs using pandas library
data = pd.read_csv(GAZE_DATA_WITH_AOIS_FILE)
# set display.max_columns to none, to show all the columns when using head()
pd.set_option('display.max_columns', None)

In [ ]:
# Preview data
data.head()

In [ ]:
#Keep only the gaze data coming from the eye-tracking device
data = data[ data['EventSource.1'] == 1 ]

In [ ]:
#Fill n/a values with -1 (for JSON Compliance in requests) 
data = data.fillna(-1) 

In [ ]:
# Oculomotor event detection (i.e., Fixations and saccades)
# Using customizable I-VT (Velocity Threshold) filter for fixation and saccade detection

dataWithEvents = None

# Iteratate over the participants and the apply I-VT filter on the data of each participant
for participant in data['Respondent'].unique():
    
    # filter in the participant data
    ParticipantData = data[data['Respondent']==participant]
    
    print(f'Applying fixation filter on the data for participant: {participant}')
    config = {
                    "fixationFilter" : "IVT",
                    "screenResolutionWidth" : 1920, 
                    "screenResolutionHeight" : 1080, 
                    "monitorSize" : 24, 
                    "screenDistance" : 60,
                    "gapFill" : True, 
                    "maxGapLength": 75, 
                    "noiseReduction" : False, 
                    "filterType" : "Median", 
                    "windowNoise" : 3,
                    "windowVelocity" : 20, 
                    "velocityThreshold" : 30, 
                    "mergeFixation" : True, 
                    "maxTimeBtwFixation" : 75,
                    "maxAngleBtwFixation" : 0.5, 
                    "discardShortFixation" : True,
                    "minDurationFixation" : 60
    }

    # POST pariticipant gaze data to the (R) eventDetection server
    url = 'http://rserver:6789/eventDetection'
    #url = 'http://127.0.0.1:6789/eventDetection'
    payload = {'config': config, 'data': ParticipantData.to_dict(orient="records")}
    response = requests.post(url, json = payload)
    
    # Create new a dataframe from the json response of the (R) eventDetection server
    participantDataWithEvents = pd.DataFrame.from_records(response.json())
    
    #Assign participantDataWithEvents to dataWithEvents or concate
    if dataWithEvents is None: 
        dataWithEvents = participantDataWithEvents
    else:
        dataWithEvents = pd.concat([dataWithEvents, participantDataWithEvents], axis=0)
        
    print('... Complete')

In [ ]:
# Preview gaze data with event information
dataWithEvents.head()

In [ ]:
# export dataWithEvents as csv
dataWithEvents.to_csv("data/gazeDataWithAOIsAndEvents.csv",  index=False)